# Sensitivity Analysis Notebook Ordinal vs Categorical

## Introduction

In this notebook, we perform a sensitivity analysis to compare the main model (ordinal) with an alternate (categorical) model.

The goal is to see if, in the future, the model will be able to handle non-ordinal comparisons such as food items.


## Libraries Used

First, we import the necessary libraries for data manipulation, probabilistic programming, and visualization.


In [1]:
# Data manipulation and analysis
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations and array manipulation

# Probabilistic programming and Bayesian statistical modeling
import pymc as pm  
import arviz as az  

# Data visualization
import matplotlib.pyplot as plt 
import seaborn as sns  

# Suppressing warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

import logging
logger = logging.getLogger("pymc")
logger.propagate = False
logger.setLevel(logging.ERROR)

## Load Preprocessed Data

### Load Cleaned Data

In [2]:
db_mon = pd.read_csv('data/mon_clean.csv')
db_med = pd.read_csv('data/med_clean.csv')

db_mon_online = pd.read_csv('data_online/mon_clean.csv')
db_med_online = pd.read_csv('data_online/med_clean.csv')

print('Participants in-person: ', len(db_mon['sub'].unique()), '\nparticipants online: ', len(db_mon_online['sub'].unique()))

Participants in-person:  66 
participants online:  332


### Extract Indices and Subject Numbers

In [3]:
# Assign a unique serial number for each participant.
db_mon['subn'] = db_mon['sub'].rank(method='dense').astype(int) - 1
db_mon_online['subn'] = db_mon_online['sub'].rank(method='dense').astype(int) - 1


# Count the number of unique subjects in the 'db_mon' dataset.
n_subs = db_mon['subn'].unique().shape[0]
n_subs_online = db_mon_online['subn'].unique().shape[0]

# Create a list of subject indices for all rows in the 'db_mon' dataset.
sub_idx = db_mon.subn.tolist()
sub_idx_online = db_mon_online.subn.tolist()

# Assign a unique serial number for each participant. This will be useful for indexing operations.
db_med['subn'] = db_med['sub'].rank(method='dense').astype(int) - 1
db_med_online['subn'] = db_med_online['sub'].rank(method='dense').astype(int) - 1


# Count the number of unique subjects in the 'db_med' dataset.
n_subs_med = db_med['subn'].unique().shape[0]
n_subs_med_online = db_med_online['subn'].unique().shape[0]

# Create a list of subject indices for all rows in the 'db_med' dataset.
sub_idx_med = db_med.subn.tolist()
sub_idx_med_online = db_med_online.subn.tolist()

## 1. Estimated Value Model (Ordinal)

In this model, we assume that the data is ordinal. While we still include the ambiguity attitude parameter, the risk attitude parameter becomes irrelevant because risk is value-based. This model is capable of estimating quantitative values for qualitative outcomes.

### Model Parameters:

- **Ambiguity Attitude (β):**
  - β = 0: Ambiguity neutral
  - β > 0: Ambiguity averse
  - β < 0: Ambiguity seeking

### Hyper Priors:
- **Ambiguity Aversion (β):** 0.65 (based on Levy et al., 2010)


We will estimate the quantitative values corresponding to these ordinal outcomes.


In [4]:
def estimate_value_ord(df, n_subs, idx):
    """
    Estimate the value of different reward levels using ordinal constraints and a common hyperprior for each level. 
    The model ensures that the levels are positive (ordinal constraints).

    Parameters:
    - df: DataFrame with trial-specific details, such as choices, value levels, risk, and ambiguity levels.
    - n_sub: Total number of subjects in the dataset.
    - idx: A list indicating the subject ID for each observation/trial.

    Returns:
    - trace: Samples from the posterior distribution of the model.
    """
    
    with pm.Model() as estimate:

        # Hyperparameters for group-level distributions
        bMu  = pm.Normal('bMu', .65, 1)     # Mean for ambiguity effect distribution
        bSig = pm.Gamma('bSig', 2, 1)       # SD for ambiguity effect distribution


       # Hyperparameters for group-level subjective value levels
        l1Mu = pm.TruncatedNormal('l1Mu', 4, 2, lower=0)  # Mean for value of level 1
        l2Mu = pm.TruncatedNormal('l2Mu', 4, 2, lower=0)  # ... level 2
        l3Mu = pm.TruncatedNormal('l3Mu', 4, 2, lower=0)  # ... level 3
        l4Mu = pm.TruncatedNormal('l4Mu', 4, 2, lower=0)  # ... level 4
        
        l1sd = pm.Gamma('l1sd', 3, 1)  # SD for value of level 1
        l2sd = pm.Gamma('l2sd', 3, 1)  # ... level 2
        l3sd = pm.Gamma('l3sd', 3, 1)  # ... level 3
        l4sd = pm.Gamma('l4sd', 3, 1)  # ... level 4
        
        # Subject-specific priors 
        β = pm.Normal('β',    bMu, bSig, shape = n_subs)   # Modulation of ambiguity effect
        γ = pm.Lognormal('γ', 0, 0.25, shape = n_subs)   # Inverse temperature, impacting choice stochasticity

        # Priors for subjective values of the different reward levels for each subject.
        level1 = pm.TruncatedNormal('level1', l1Mu, l1sd, lower = 0, shape = n_subs)
        level2 = pm.TruncatedNormal('level2', l2Mu, l2sd, lower = 0, shape = n_subs)
        level3 = pm.TruncatedNormal('level3', l3Mu, l3sd, lower = 0, shape = n_subs)
        level4 = pm.TruncatedNormal('level4', l4Mu, l4sd, lower = 0, shape = n_subs)

        # Calculate the total expected value for each trial by combining values from different levels
        val = (df['l1'].values * level1[idx] + 
               df['l2'].values * level2[idx] + 
               df['l3'].values * level3[idx] + 
               df['l4'].values * level4[idx]) 

        # Calculate adjusted probability by considering both risk and ambiguity levels modulated by β
        prob = (df['risk'].values) - (β[idx] * (df['ambiguity'].values/2))  

        # Compute the subjective value of the lottery option
        svLotto = val * prob
        svRef   = level1[idx]  # The subjective value of the reference option

        # Transform the SV difference between lottery and reference into a choice probability using the logistic function
        p  = (svLotto - svRef) / γ[idx]
        mu = pm.invlogit(p)

        # Likelihood of the observed choices given the computed probabilities
        choice = pm.Binomial('choice', 1, mu, observed=df['choice'])

        trace = pm.sample(idata_kwargs={'log_likelihood':True})
        
    return(trace)

## 2. Estimated Value Model (Categorical)

In this model, we assume that the data is categorical and can hold negative values

In [5]:
def estimate_value(df, n_subs, idx):
    """
    Estimate the value of different reward levels using ordinal constraints and a common hyperprior for each level. 
    The model ensures that the levels are positive (ordinal constraints).

    Parameters:
    - df: DataFrame with trial-specific details, such as choices, value levels, risk, and ambiguity levels.
    - n_sub: Total number of subjects in the dataset.
    - idx: A list indicating the subject ID for each observation/trial.

    Returns:
    - trace: Samples from the posterior distribution of the model.
    """
    
    with pm.Model() as estimate:

        # Hyperparameters for group-level distributions
        bMu  = pm.Normal('bMu', .65, 1)     # Mean for ambiguity effect distribution
        bSig = pm.Gamma('bSig', 2, 1)       # SD for ambiguity effect distribution


       # Hyperparameters for group-level subjective value levels
        l1Mu = pm.Normal('l1Mu', 4, 2)  # Mean for value of level 1
        l2Mu = pm.Normal('l2Mu', 4, 2)  # ... level 2
        l3Mu = pm.Normal('l3Mu', 4, 2)  # ... level 3
        l4Mu = pm.Normal('l4Mu', 4, 2)  # ... level 4
        
        l1sd = pm.Gamma('l1sd', 3, 1)  # SD for value of level 1
        l2sd = pm.Gamma('l2sd', 3, 1)  # ... level 2
        l3sd = pm.Gamma('l3sd', 3, 1)  # ... level 3
        l4sd = pm.Gamma('l4sd', 3, 1)  # ... level 4
        
        # Subject-specific priors 
        β = pm.Normal('β',    bMu, bSig, shape = n_subs)   # Modulation of ambiguity effect
        γ = pm.Lognormal('γ', 0, 0.25, shape = n_subs)   # Inverse temperature, impacting choice stochasticity

        # Priors for subjective values of the different reward levels for each subject.
        level1 = pm.Normal('level1', l1Mu, l1sd, shape = n_subs)
        level2 = pm.Normal('level2', l2Mu, l2sd, shape = n_subs)
        level3 = pm.Normal('level3', l3Mu, l3sd, shape = n_subs)
        level4 = pm.Normal('level4', l4Mu, l4sd, shape = n_subs)

        # Calculate the total expected value for each trial by combining values from different levels
        val = (df['l1'].values * level1[idx] + 
               df['l2'].values * level2[idx] + 
               df['l3'].values * level3[idx] + 
               df['l4'].values * level4[idx]) 

        # Calculate adjusted probability by considering both risk and ambiguity levels modulated by β
        prob = (df['risk'].values) - (β[idx] * (df['ambiguity'].values/2))  

        # Compute the subjective value of the lottery option
        svLotto = val * prob
        svRef   = level1[idx]  # The subjective value of the reference option

        # Transform the SV difference between lottery and reference into a choice probability using the logistic function
        p  = (svLotto - svRef) / γ[idx]
        mu = pm.invlogit(p)

        # Likelihood of the observed choices given the computed probabilities
        choice = pm.Binomial('choice', 1, mu, observed=df['choice'])

        trace = pm.sample(idata_kwargs={'log_likelihood':True})
        
    return(trace)

## Run the models on the in-person datasets

In [6]:
mon_estimated = estimate_value(db_mon, n_subs, sub_idx)
med_estimated = estimate_value(db_med, n_subs_med, sub_idx_med)

mon_estimated_ord = estimate_value_ord(db_mon, n_subs, sub_idx)
med_estimated_ord = estimate_value_ord(db_med, n_subs_med, sub_idx_med)

## Run the models on the online datasets

In [7]:
mon_estimated_online = estimate_value(db_mon_online, n_subs_online, sub_idx_online)
med_estimated_online = estimate_value(db_med_online, n_subs_med_online, sub_idx_med_online)

mon_estimated_online_ord = estimate_value_ord(db_mon_online, n_subs_online, sub_idx_online)
med_estimated_online_ord = estimate_value_ord(db_med_online, n_subs_med_online, sub_idx_med_online)

## Compare models

In [8]:
print('Monetary in-Person')
compare_dict = {'Estimate Value':  mon_estimated,
                'Estimate Value Ordinal': mon_estimated_ord

}

comp = az.compare(compare_dict)
comp

Monetary in-Person


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
Estimate Value Ordinal,0,-1562.699592,225.175570,0.000000,0.867314,45.704947,0.000000,True,log
Estimate Value,1,-1568.582929,241.041979,5.883337,0.132686,45.507016,4.031194,True,log


In [9]:
print('Medical in-Person')
compare_dict = {'Estimate Value':  med_estimated,
                'Estimate Value Ordinal': med_estimated_ord

}

comp = az.compare(compare_dict)
comp

Medical in-Person


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
Estimate Value Ordinal,0,-1412.914788,211.109329,0.000000,0.7293,45.029311,0.000000,True,log
Estimate Value,1,-1417.008574,227.235556,4.093785,0.2707,44.865177,4.285933,True,log


In [10]:
print('Monetary on-line')
compare_dict = {'Estimate Value':  mon_estimated_online,
                'Estimate Value Ordinal': mon_estimated_online_ord

}

comp = az.compare(compare_dict)
comp

Monetary on-line


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
Estimate Value Ordinal,0,-3789.546667,824.017288,0.000000,0.953365,69.079413,0.000000,True,log
Estimate Value,1,-3816.660409,895.615392,27.113742,0.046635,69.074977,7.709978,True,log


In [11]:
print('Medical On-Line')
compare_dict = {'Estimate Value':  med_estimated_online,
                'Estimate Value Ordinal': med_estimated_online_ord

}

comp = az.compare(compare_dict)
comp

Medical On-Line


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
Estimate Value Ordinal,0,-4097.333596,782.348055,0.000000,0.589611,72.532839,0.000000,True,log
Estimate Value,1,-4105.425944,870.597374,8.092348,0.410389,72.238026,10.180671,True,log


## Extract Estimated Values

In [12]:
print("In-person: ")

print("level 1 (med): mean:", round(az.summary(med_estimated, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level1'])['sd'].mean(),2))
print("level 2 (med): mean:", round(az.summary(med_estimated, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level2'])['sd'].mean(),2))
print("level 3 (med): mean:", round(az.summary(med_estimated, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level3'])['sd'].mean(),2))
print("level 4 (med): mean:", round(az.summary(med_estimated, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level4'])['sd'].mean(),2))
print()
print("level 1 (mon): mean:", round(az.summary(mon_estimated, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level1'])['sd'].mean(),2))
print("level 2 (mon): mean:", round(az.summary(mon_estimated, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level2'])['sd'].mean(),2))
print("level 3 (mon): mean:", round(az.summary(mon_estimated, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level3'])['sd'].mean(),2))
print("level 4 (mon): mean:", round(az.summary(mon_estimated, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level4'])['sd'].mean(),2))

In-person: 
level 1 (med): mean: 6.84 SD:  1.64
level 2 (med): mean: 8.86 SD:  2.34
level 3 (med): mean: 6.94 SD:  2.93
level 4 (med): mean: 3.87 SD:  3.06

level 1 (mon): mean: 7.0 SD:  1.56
level 2 (mon): mean: 3.89 SD:  1.67
level 3 (mon): mean: 6.11 SD:  2.32
level 4 (mon): mean: 8.25 SD:  3.36


In [13]:
print("Online: ")

print("level 1 (med): mean:", round(az.summary(med_estimated_online, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level1'])['sd'].mean(),2))
print("level 2 (med): mean:", round(az.summary(med_estimated_online, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level2'])['sd'].mean(),2))
print("level 3 (med): mean:", round(az.summary(med_estimated_online, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level3'])['sd'].mean(),2))
print("level 4 (med): mean:", round(az.summary(med_estimated_online, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level4'])['sd'].mean(),2))
print()
print("level 1 (mon): mean:", round(az.summary(mon_estimated_online, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level1'])['sd'].mean(),2))
print("level 2 (mon): mean:", round(az.summary(mon_estimated_online, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level2'])['sd'].mean(),2))
print("level 3 (mon): mean:", round(az.summary(mon_estimated_online, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level3'])['sd'].mean(),2))
print("level 4 (mon): mean:", round(az.summary(mon_estimated_online, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level4'])['sd'].mean(),2))

Online: 
level 1 (med): mean: 8.49 SD:  2.3
level 2 (med): mean: 11.95 SD:  3.63
level 3 (med): mean: 4.51 SD:  2.91
level 4 (med): mean: 2.35 SD:  2.45

level 1 (mon): mean: 10.92 SD:  2.68
level 2 (mon): mean: 3.93 SD:  2.04
level 3 (mon): mean: 5.29 SD:  2.37
level 4 (mon): mean: 3.57 SD:  2.57


In [14]:
print("In-person: ")

print("level 1 (med): mean:", round(az.summary(med_estimated_ord, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level1'])['sd'].mean(),2))
print("level 2 (med): mean:", round(az.summary(med_estimated_ord, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level2'])['sd'].mean(),2))
print("level 3 (med): mean:", round(az.summary(med_estimated_ord, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level3'])['sd'].mean(),2))
print("level 4 (med): mean:", round(az.summary(med_estimated_ord, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level4'])['sd'].mean(),2))
print()
print("level 1 (mon): mean:", round(az.summary(mon_estimated_ord, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level1'])['sd'].mean(),2))
print("level 2 (mon): mean:", round(az.summary(mon_estimated_ord, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level2'])['sd'].mean(),2))
print("level 3 (mon): mean:", round(az.summary(mon_estimated_ord, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level3'])['sd'].mean(),2))
print("level 4 (mon): mean:", round(az.summary(mon_estimated_ord, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level4'])['sd'].mean(),2))

In-person: 
level 1 (med): mean: 6.92 SD:  1.64
level 2 (med): mean: 8.96 SD:  2.34
level 3 (med): mean: 7.02 SD:  2.93
level 4 (med): mean: 4.17 SD:  3.06

level 1 (mon): mean: 7.21 SD:  1.56
level 2 (mon): mean: 4.13 SD:  1.67
level 3 (mon): mean: 6.23 SD:  2.32
level 4 (mon): mean: 8.77 SD:  3.36


In [15]:
print("Online: ")

print("level 1 (med): mean:", round(az.summary(med_estimated_online_ord, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level1'])['sd'].mean(),2))
print("level 2 (med): mean:", round(az.summary(med_estimated_online_ord, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level2'])['sd'].mean(),2))
print("level 3 (med): mean:", round(az.summary(med_estimated_online_ord, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level3'])['sd'].mean(),2))
print("level 4 (med): mean:", round(az.summary(med_estimated_online_ord, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level4'])['sd'].mean(),2))
print()
print("level 1 (mon): mean:", round(az.summary(mon_estimated_online_ord, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level1'])['sd'].mean(),2))
print("level 2 (mon): mean:", round(az.summary(mon_estimated_online_ord, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level2'])['sd'].mean(),2))
print("level 3 (mon): mean:", round(az.summary(mon_estimated_online_ord, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level3'])['sd'].mean(),2))
print("level 4 (mon): mean:", round(az.summary(mon_estimated_online_ord, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level4'])['sd'].mean(),2))

Online: 
level 1 (med): mean: 8.62 SD:  2.3
level 2 (med): mean: 12.63 SD:  3.63
level 3 (med): mean: 4.66 SD:  2.91
level 4 (med): mean: 2.38 SD:  2.45

level 1 (mon): mean: 10.91 SD:  2.68
level 2 (mon): mean: 4.13 SD:  2.04
level 3 (mon): mean: 5.05 SD:  2.37
level 4 (mon): mean: 3.92 SD:  2.57


### Difference in Values

In [16]:

print("level 1 (med):", round(az.summary(med_estimated, var_names=['level1'])['mean'].mean(),2) - round(az.summary(med_estimated_ord, var_names=['level1'])['mean'].mean(),2))
print("level 2 (med):", round(az.summary(med_estimated, var_names=['level2'])['mean'].mean(),2) - round(az.summary(med_estimated_ord, var_names=['level2'])['mean'].mean(),2))
print("level 3 (med):", round(az.summary(med_estimated, var_names=['level3'])['mean'].mean(),2) - round(az.summary(med_estimated_ord, var_names=['level3'])['mean'].mean(),2))
print("level 4 (med):", round(az.summary(med_estimated, var_names=['level4'])['mean'].mean(),2) - round(az.summary(med_estimated_ord, var_names=['level4'])['mean'].mean(),2))
print()
print("level 1 (mon):", round(az.summary(mon_estimated, var_names=['level1'])['mean'].mean(),2) - round(az.summary(mon_estimated_ord, var_names=['level1'])['mean'].mean(),2))
print("level 2 (mon):", round(az.summary(mon_estimated, var_names=['level2'])['mean'].mean(),2) - round(az.summary(mon_estimated_ord, var_names=['level2'])['mean'].mean(),2))
print("level 3 (mon):", round(az.summary(mon_estimated, var_names=['level3'])['mean'].mean(),2) - round(az.summary(mon_estimated_ord, var_names=['level3'])['mean'].mean(),2))
print("level 4 (mon):", round(az.summary(mon_estimated, var_names=['level4'])['mean'].mean(),2) - round(az.summary(mon_estimated_ord, var_names=['level4'])['mean'].mean(),2))

level 1 (med): -0.08000000000000007
level 2 (med): -0.10000000000000142
level 3 (med): -0.07999999999999918
level 4 (med): -0.2999999999999998

level 1 (mon): -0.20999999999999996
level 2 (mon): -0.23999999999999977
level 3 (mon): -0.1200000000000001
level 4 (mon): -0.5199999999999996


In [17]:

print("level 1 (med):", round(az.summary(med_estimated_online, var_names=['level1'])['mean'].mean(),2) - round(az.summary(med_estimated_online_ord, var_names=['level1'])['mean'].mean(),2))
print("level 2 (med):", round(az.summary(med_estimated_online, var_names=['level2'])['mean'].mean(),2) - round(az.summary(med_estimated_online_ord, var_names=['level2'])['mean'].mean(),2))
print("level 3 (med):", round(az.summary(med_estimated_online, var_names=['level3'])['mean'].mean(),2) - round(az.summary(med_estimated_online_ord, var_names=['level3'])['mean'].mean(),2))
print("level 4 (med):", round(az.summary(med_estimated_online, var_names=['level4'])['mean'].mean(),2) - round(az.summary(med_estimated_online_ord, var_names=['level4'])['mean'].mean(),2))
print()
print("level 1 (mon):", round(az.summary(mon_estimated_online, var_names=['level1'])['mean'].mean(),2) - round(az.summary(mon_estimated_online_ord, var_names=['level1'])['mean'].mean(),2))
print("level 2 (mon):", round(az.summary(mon_estimated_online, var_names=['level2'])['mean'].mean(),2) - round(az.summary(mon_estimated_online_ord, var_names=['level2'])['mean'].mean(),2))
print("level 3 (mon):", round(az.summary(mon_estimated_online, var_names=['level3'])['mean'].mean(),2) - round(az.summary(mon_estimated_online_ord, var_names=['level3'])['mean'].mean(),2))
print("level 4 (mon):", round(az.summary(mon_estimated_online, var_names=['level4'])['mean'].mean(),2) - round(az.summary(mon_estimated_online_ord, var_names=['level4'])['mean'].mean(),2))

level 1 (med): -0.129999999999999
level 2 (med): -0.6800000000000015
level 3 (med): -0.15000000000000036
level 4 (med): -0.029999999999999805

level 1 (mon): 0.009999999999999787
level 2 (mon): -0.19999999999999973
level 3 (mon): 0.2400000000000002
level 4 (mon): -0.3500000000000001


In [18]:
%load_ext watermark
%watermark -n -u -v -iv -w -p xarray

Last updated: Mon Oct 14 2024

Python implementation: CPython
Python version       : 3.10.14
IPython version      : 8.26.0

xarray: 2024.7.0

pymc      : 4.1.7
logging   : 0.5.1.2
matplotlib: 3.6.3
numpy     : 1.23.5
seaborn   : 0.12.2
pandas    : 1.5.3
arviz     : 0.17.1

Watermark: 2.4.3

